# Model Training Script
This script extracts data from the snowflake table, transforms the features, and then traings the model on the engeneered features. Our training script leverages MLflow for model logging.

In [1]:
import os
from datetime import datetime
import json
import argparse
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import snowflake.connector

from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
#import functions from helper_functions.py
from helper_functions import get_data, write_to_snowflake
# Enable pandas to display up to 500 columns
pd.set_option('display.max_columns', 500)

import sys
import os

# Get the current working directory
current_directory = os.getcwd()

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(current_directory, '..')))

# Import the helper_functions module
from helper_functions import connect_to_snowflake, get_data, write_to_snowflake

## 1. Connect to Snowflake using schema ODS

In [2]:
conn = connect_to_snowflake(schema_name='ODS')

Successfully connected to Snowflake schema ODS


#### Extract data for all markets

In [3]:
sql_query = '''
select * from ods.listings
 '''

df_raw = get_data(sql_query, conn=conn)
df_raw.shape

(111052, 68)

In [5]:
# Select features and target for the model
features = ['market', 'room_type', 'accommodates', 'bathrooms', 'beds', 'latitude', 'longitude', 'amenities']
categorical_features = ['market', 'room_type']  # Features with categorical data
numerical_features = ['accommodates', 'bathrooms', 'beds', 'latitude', 'longitude']  # Features with numerical data
text_features = ['amenities']  # (Optional) Feature for text data like amenities
target = 'price'  # Target variable to predict

# Filter the dataframe to include only the selected features and target column
df = df_raw[features + [target]]

# Remove NA values from the 'price' column
df_raw_no_na = df_raw[target].dropna()
# Remove rows where the price exceeds $600 per night (97th percentile) to avoid outliers, consider dropping vals <40 (3rd percentile)
df = df[df[target] <= 600]

# Drop rows with missing values in the target column to ensure data integrity
df = df.dropna(subset=[target])
# write cleaned data to snowflake feature store
write_to_snowflake(df,conn=conn, snowflake_schema_name='FEATURE_STORE', snowflake_table_name='listings_cleaned')

In [6]:
#add code for pulling from snowflake feature store

# Separate the features (X) and the target (y)
X = df.drop(columns=[target])
y = df[target]

# Split the data into training and test sets to evaluate model performance. Note because we have a large dataset, 111,052 obsevations, we can use a smaller test size of 10% the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
# Get model run start time
start_time = datetime.now()

# Define the pipeline
def createPipeline(numerical_features, categorical_features):
    # Define the preprocessing for numerical features
    numericalTransformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Define the preprocessing for categorical features
    categoricalTransformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine preprocessing for numerical and categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numericalTransformer, numerical_features),
            ('cat', categoricalTransformer, categorical_features)
        ])
    
    # Create the pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(max_depth=30,
                                            max_features=None,
                                            min_samples_leaf=2,
                                            min_samples_split=2,
                                            n_estimators=300,
                                            random_state=42)) #best model after hyperparameter tuning
    ])
    return pipeline

def log_to_file(log_file, params, metrics, training_details):
    # Load existing data
    try:
        with open(log_file, 'r') as f:
            logs = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        logs = []

    # Append new log entry
    log_entry = {
        "params": params,
        "metrics": metrics,
        "training_details": training_details
    }
    logs.append(log_entry)

    # Write updated logs back to the file
    with open(log_file, 'w') as f:
        json.dump(logs, f, indent=4)

# Parameters and file paths
log_file = 'model/experiment_log.json'
model_file = 'model/regression_pipeline.joblib'

# Create the pipeline
pipeline = createPipeline(numerical_features, categorical_features)

# Perform cross-validation with Mean Absolute Error (MAE)
maeScores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)
maeScores = -maeScores  # Convert negative MAE to positive
print("Cross-validation MAE scores:", maeScores)
print("Average cross-validation MAE:", maeScores.mean())

# Perform cross-validation with Root Mean Squared Error (RMSE)
rmse_scorer = make_scorer(mean_squared_error, squared=False)  # squared=False returns RMSE
rmseScores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring=rmse_scorer, n_jobs=-1)
print("Cross-validation RMSE scores:", rmseScores)
print("Average cross-validation RMSE:", rmseScores.mean())

# Parameters and metrics to log
params = {
    "model_type": str(pipeline.get_params()['regressor']),
    "numerical_features": numerical_features,
    "categorical_features": categorical_features
}
metrics = {
    "cv_mae_mean": maeScores.mean(),
    "cv_rmse_mean": rmseScores.mean(),
    "cv_mae_std": maeScores.std(),
    "cv_rmse_std": rmseScores.std()
}
training_details = {
    "train_size": len(X_train),
    "test_size": len(X_test),
    "train_duration": str(datetime.now() - start_time)
}

# Train the model on the entire training set
pipeline.fit(X_train, y_train)

# Predict and evaluate the model on the test set using MAE
y_pred = pipeline.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred)
print("Test set MAE:", mae_test)

# Calculate RMSE on the test set
rmse_test = mean_squared_error(y_test, y_pred, squared=False)  # squared=False returns RMSE
print("Test set RMSE:", rmse_test)

# Log training run results to file
metrics.update({"test_mae": mae_test, "test_rmse": rmse_test})
#log_to_file(log_file, params, metrics, training_details)

# Refit the final model on the entire dataset to improve generalization by leveraging all available information
pipeline.fit(X, y)

# Save the model
joblib.dump(pipeline, model_file, compress=('gzip', 9))


Cross-validation MAE scores: [46.92844899 45.71262102 46.13681104 46.18273602 46.38888795 46.11667545
 46.19433292 46.97021873 46.64408421 46.03438575]
Average cross-validation MAE: 46.330920206389905
Cross-validation RMSE scores: [70.48089713 69.07791366 69.85507162 70.81930847 70.48595348 69.76894344
 69.36436791 71.07897349 71.5754946  69.38603469]
Average cross-validation RMSE: 70.1892958498672
Test set MAE: 46.16088187131708
Test set RMSE: 69.43573853209195


c:\Venv\analytics\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


['model/regression_pipeline.joblib']